### COVID-19 DATASET HANDLING SECTION

In [1]:
# pip install pydicom

In [2]:
import pydicom
import os
import pandas as pd

In [9]:
data_dir = '../Dataset/CSV/'
patients = os.listdir(data_dir)
labels_df = pd.read_csv('../Dataset/CSV/Dataset_Labels_Summary.csv', index_col=1)

labels_df.head()

,Unnamed: 0,Label_1_Virus_category,Label_2_Virus_category,Image_Count
Label,,,,
Normal,0,NaN,NaN,1576
Pnemonia,1,Stress-Smoking,ARDS,2
Pnemonia,2,Virus,NaN,1493
Pnemonia,3,Virus,COVID-19,58
Pnemonia,4,Virus,SARS,4


In [4]:
import cv2
import numpy as np
import math
import matplotlib.pyplot as plt

In [5]:
IMG_PX_SIZE = 50
HM_SLICES = 20

In [6]:
def Chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i+n]

In [7]:
def Mean(l):
    return sum(l)/len(l)

In [8]:
def ProcessData(patient, labels_df, img_px_size = 50, hm_slices = 20, visualize = False):
    label = label_df.get_value(patient, 'covid')
    path = data_dir + patient